http://deeplearning.net/software/theano/library/tensor/basic.html

http://sinhrks.hatenablog.com/entry/2014/11/26/002818

日本語解説はhttp://aidiary.hatenablog.com/entry/20150509/1431137590

公式チュートリアルは
http://deeplearning.net/software/theano/tutorial/index.html

In [2]:
import theano.tensor as T
x = T.fmatrix()
x.shape

Shape.0

In [3]:
x = T.iscalar('myvar')

In [ ]:
import numpy
y=numpy.asarray([1,2])
y2=numpy.array([1,2])

from theano import function

If you provide no argument, the symbol will be unnamed. Names are not required, but they can help debugging.

In [8]:
x = T.dscalar('x')
y = T.dscalar('y')
z = x + y

この段階ではまだ関数になっていない。以下はエラーとなる

In [9]:
z(2,3)

TypeError: 'TensorVariable' object is not callable

以下のようにして関数を定義する

In [18]:
f = function([x,y],z) #more flexible function definition, maybe preferrable consequnently
print(f(2,3)) # function defined as f eats x,y,z,...
print(f(5,5))
print(type(f(5,5)))
print((f(5,5)).shape)

5.0
10.0
<class 'numpy.ndarray'>
()


結果の照合例：

In [19]:
numpy.allclose(f(2,3),5)

True

theano.functionを用いない関数定義は以下のようにする（？）

evalはdict型を食うことに注意

In [21]:
z2 = x + y
z2.eval({x : 2 , y : 3}) #z eats a dict type input through eval

array(5.0)

In [27]:
print(type(x.eval({x:1})))
tmp = {x : 2 , y : 3}
print(type(tmp))
tmp[x]
print(z2.eval(tmp))

<class 'numpy.ndarray'>
<class 'dict'>
5.0


dictは名前もきちんとmatchしている必要がある：

In [29]:
p = T.dscalar('p0')
q = T.dscalar('q0')
r = p + q
r.eval({p:1 , q :1})

array(2.0)

In [30]:
r.eval({p0:1 , q0 :1}) #error

NameError: name 'p0' is not defined

In [31]:
r.eval({"p0":1 , "q0" :1}) #error

TypeError: Unknown parameter type: <class 'str'>

In [36]:
p = T.dscalar()
q = T.dscalar()
r = p + 2 * q
f2 = function([p , q] , r)
f2(1,2)

array(5.0)

In [34]:
r.eval({p:1 , q :2})

array(5.0)

In [38]:
T.printing.debugprint(r)

Elemwise{add,no_inplace} [id A] ''   
 |<TensorType(float64, scalar)> [id B]
 |Elemwise{mul,no_inplace} [id C] ''   
   |TensorConstant{2} [id D]
   |<TensorType(float64, scalar)> [id E]


In [39]:
T.printing.debugprint(f2)

Elemwise{Composite{(i0 + (i1 * i2))}} [id A] ''   0
 |<TensorType(float64, scalar)> [id B]
 |TensorConstant{2.0} [id C]
 |<TensorType(float64, scalar)> [id D]


2次元のアウトプットを返す例：

In [41]:
s = T.dscalar('s0')
t = r + s
u = function([p , q , s]  , [r , t])

返り値はlist:

In [43]:
print(u(1,2,3))
print(type(u(1,2,3)))

[array(5.0), array(8.0)]
<class 'list'>


行列を引数として取る例：

In [45]:
x = T.dmatrix()
s = 1 / (1 + T.exp(-x))# expはcomponentwise
s.eval({x:[[1]]})

array([[ 0.73105858]])

In [46]:
s.eval({x:[[1 , 2] , [3,4]]})

array([[ 0.73105858,  0.88079708],
       [ 0.95257413,  0.98201379]])

In [ ]:
T.printing.debugprint(s)

より読みやすい関数のサマリーの返し方：

In [47]:
from theano import pp#pretty print
pp(s)

'(TensorConstant{1} / (TensorConstant{1} + exp((-<TensorType(float64, matrix)>))))'

shared variableの使用例は以下。
sharedとして宣言された変数は内部状態として破棄されず保存される。

In [55]:
from theano import shared
state = shared(1 , name='state')

In [62]:
state.get_value()

array(437)

In [ ]:
inc = T.iscalar('inc')

In [ ]:
inc = T.iscalar('inc')
accumulator = function(inputs = [inc] , outputs = [state] , updates=[(state  , state + inc)])#update happens after output evaluation

上記のように定義すると、下記は評価するたびに（stateが変化するので）違う値を返す：

In [61]:
accumulator(109)

[array(328)]